In [1]:
import sys
import pymc3 as pm

sys.path.insert(0, "../")
from kshell_for_pymc3 import *
from kshell_settings import KSHELL_DIR, IS_MPI

In [2]:
# === Set up shell model ensemble ===
truncation = [[],[]] # Truncations of the form [[orbital_numbers], [min_part1, max_part1, min_part2, max_part2, ...]]
core = {'A': 4, 'Z': 2}
model_space = {1: {'j': 1, 'Tz': -1, 'l': 1, 'n': 0}, 
               2: {'j': 3, 'Tz': -1, 'l': 1, 'n': 0}, 
               3: {'j': 1, 'Tz':  1, 'l': 1, 'n': 0}, 
               4: {'j': 3, 'Tz':  1, 'l': 1, 'n': 0}}

spe_template = np.array([ 2.419,  1.129,  2.419,  1.129])
tbme_template = np.array([[ 1.     ,  1.     ,  1.     ,  1.     ,  0.     ,  0.244  ],
     [ 1.     ,  1.     ,  2.     ,  2.     ,  0.     , -5.0526 ],
     [ 1.     ,  2.     ,  1.     ,  2.     ,  1.     ,  0.7344 ],
     [ 1.     ,  2.     ,  1.     ,  2.     ,  2.     , -1.1443 ],
     [ 1.     ,  2.     ,  2.     ,  2.     ,  2.     ,  1.7423 ],
     [ 2.     ,  2.     ,  2.     ,  2.     ,  0.     , -3.3287 ],
     [ 2.     ,  2.     ,  2.     ,  2.     ,  2.     ,  0.0878 ],
     [ 3.     ,  3.     ,  3.     ,  3.     ,  0.     ,  0.244  ],
     [ 3.     ,  3.     ,  4.     ,  4.     ,  0.     , -5.0526 ],
     [ 3.     ,  4.     ,  3.     ,  4.     ,  1.     ,  0.7344 ],
     [ 3.     ,  4.     ,  3.     ,  4.     ,  2.     , -1.1443 ],
     [ 3.     ,  4.     ,  4.     ,  4.     ,  2.     ,  1.7423 ],
     [ 4.     ,  4.     ,  4.     ,  4.     ,  0.     , -3.3287 ],
     [ 4.     ,  4.     ,  4.     ,  4.     ,  2.     ,  0.0878 ],
     [ 1.     ,  3.     ,  1.     ,  3.     ,  0.     ,  0.244  ],
     [ 1.     ,  3.     ,  1.     ,  3.     ,  1.     , -4.29215],
     [ 1.     ,  3.     ,  1.     ,  4.     ,  1.     , -0.85185],
     [ 1.     ,  3.     ,  2.     ,  3.     ,  1.     ,  0.85185],
     [ 1.     ,  3.     ,  2.     ,  4.     ,  0.     , -5.0526 ],
     [ 1.     ,  3.     ,  2.     ,  4.     ,  1.     ,  1.7698 ],
     [ 1.     ,  4.     ,  1.     ,  4.     ,  1.     , -2.91415],
     [ 2.     ,  3.     ,  1.     ,  4.     ,  1.     ,  3.64855],
     [ 2.     ,  3.     ,  2.     ,  3.     ,  1.     , -2.91415],
     [ 1.     ,  4.     ,  1.     ,  4.     ,  2.     , -2.6011 ],
     [ 2.     ,  3.     ,  1.     ,  4.     ,  2.     , -1.4568 ],
     [ 2.     ,  3.     ,  2.     ,  3.     ,  2.     , -2.6011 ],
     [ 1.     ,  4.     ,  2.     ,  4.     ,  1.     , -2.2667 ],
     [ 2.     ,  3.     ,  2.     ,  4.     ,  1.     ,  2.2667 ],
     [ 1.     ,  4.     ,  2.     ,  4.     ,  2.     ,  1.23199],
     [ 2.     ,  3.     ,  2.     ,  4.     ,  2.     , -1.23199],
     [ 2.     ,  4.     ,  2.     ,  4.     ,  0.     , -3.3287 ],
     [ 2.     ,  4.     ,  2.     ,  4.     ,  1.     , -3.4362 ],
     [ 2.     ,  4.     ,  2.     ,  4.     ,  2.     ,  0.0878 ],
     [ 2.     ,  4.     ,  2.     ,  4.     ,  3.     , -7.2668 ]])




# The tbme_template needs to be sorted before input in order for the gradient indices to be right
tbme_template = tbme_template[np.lexsort((tbme_template[:,4],tbme_template[:,3],tbme_template[:,2],tbme_template[:,1],tbme_template[:,0]))]
nucleiList = { 
  # This list of nuclei is generated using the extract_level_info_from_RIPL.py script to get spin, parity information. 
  # The selection is based on manually going throught ENSDF entries,
  # stopping before unnatural-parity or undetermined levels
   "B10": {"A":10, "Z":5, "levels":["3.0+2","0.0+1","1.0+2","2.0+1",], "truncation":truncation},
  # "B11": {"A":11, "Z":5, "levels":["3.5-1","0.5-1","2.5-1","1.5-2",], "truncation":truncation},
  # "B12": {"A":12, "Z":5, "levels":["1.0+1","2.0+1",], "truncation":truncation},
  # "B9": {"A":9, "Z":5, "levels":["1.5-1",], "truncation":truncation},
  # "Be10": {"A":10, "Z":4, "levels":["0.0+1","2.0+2",], "truncation":truncation},
  # "Be6": {"A":6, "Z":4, "levels":["0.0+1",], "truncation":truncation},
  # "Be7": {"A":7, "Z":4, "levels":["3.5-2","0.5-2","2.5-2","1.5-3",], "truncation":truncation},
  # "Be8": {"A":8, "Z":4, "levels":["0.0+1","1.0+2","2.0-1","4.0+1","2.0+3",], "truncation":truncation},
  # "Be9": {"A":9, "Z":4, "levels":["2.5+1","0.5+1","0.5-1","2.5-1","1.5-1",], "truncation":truncation},
  # "C10": {"A":10, "Z":6, "levels":["0.0+1","2.0+1",], "truncation":truncation},
  # "C11": {"A":11, "Z":6, "levels":["0.5-1","2.5-1","1.5-2",], "truncation":truncation},
  # "C12": {"A":12, "Z":6, "levels":["0.0+2","2.0+1",], "truncation":truncation},
  # "C13": {"A":13, "Z":6, "levels":["0.5-1",], "truncation":truncation},
  # "C8": {"A":8, "Z":6, "levels":["0.0+1",], "truncation":truncation},
  # "He5": {"A":5, "Z":2, "levels":["1.5-1",], "truncation":truncation},
  # "He6": {"A":6, "Z":2, "levels":["0.0+1","2.0+1",], "truncation":truncation},
  "Li5": {"A":5, "Z":3, "levels":["1.5-1",], "truncation":truncation},
  # "Li6": {"A":6, "Z":3, "levels":["3.0+2","0.0+1","1.0+2","2.0+2",], "truncation":truncation},
  # "Li7": {"A":7, "Z":3, "levels":["3.5-2","0.5-1","2.5-2","1.5-3",], "truncation":truncation},
  # "Li8": {"A":8, "Z":3, "levels":["3.0+1","1.0+3","2.0+1",], "truncation":truncation},
  # "Li9": {"A":9, "Z":3, "levels":["1.5-1",], "truncation":truncation},
  # "N12": {"A":12, "Z":7, "levels":["1.0+1","2.0+1",], "truncation":truncation},
  # "N13": {"A":13, "Z":7, "levels":["0.5-1",], "truncation":truncation},
   "N14": {"A":14, "Z":7, "levels":["0.0+1","1.0+2",], "truncation":truncation},
}
from collections import OrderedDict
nucleiList = OrderedDict(sorted(nucleiList.items()))

# Sum number of levels in all calculations
calclevelsum = 0
for nucleusName, nucleusAttr in nucleiList.items():
  calclevelsum += sum_num_levels(nucleusAttr["levels"])
print("Total number of levels for all SM calculations N = {:d}".format(calclevelsum))

# E_exp = np.array([-3.944, -3.061, -11.787, -9.764, -8.432, -7.636, -7.514,]) # O17 + O18 USDa values, for testing
expLevelPath = "../generated_comparison_data/p_shell"
expLevelsDict = {}
# E_exp = []
for nucleusName in list(nucleiList.keys()):
  filename = os.path.join(expLevelPath, nucleusName, "summary_"+nucleusName+".txt")
  levels = smutil.read_energy_levels(filename)
  Egs = levels[0,0] 
  levels[:,0] = levels[:,0]-Egs
  # E_exp = 
  expLevelsDict[nucleusName] = {"levels":levels}
  expLevelsDict[nucleusName]["Egs"] = Egs
  # expLevelsDict[nucleusName]["sigma"] = 1e-3*np.ones(len(levels[:,0])) # Take sigma to be 1 keV for calculation uncert, as a first test
  expLevelsDict[nucleusName]["sigma"] = 1*np.ones(len(levels[:,0])) 
# sigma = np.ones(len(E_exp))
A0 = 4 # Mass scaling parameter
p = -0.3 # Mass scaling parameter

Total number of levels for all SM calculations N = 10


In [3]:
# === Test chisquare function: ===
th.config.exception_verbosity = "high"
variable = th.tensor.dvector()
tt_chisquare = ChiSquare(core, model_space, spe_template, tbme_template, nucleiList, KSHELL_DIR,
                         expLevelsDict, mass_scaling=True, scaling_A0=A0, scaling_p=p, is_mpi=IS_MPI)
f_tt_chisquare = th.function([variable], tt_chisquare(variable))

# Evaluate the chisquare at the exact parameter point that was used to generated the synthetic data:
print("Chisquare value at true parameter point (should be zero):\n", f_tt_chisquare(np.hstack((spe_template, tbme_template[:,5]))))

Chisquare value at true parameter point (should be zero):
 0.0


In [4]:
# # === Test gradient -- 
trial_vector = 0.9*np.hstack((spe_template, tbme_template[:,5]))
print("trial_vector =", trial_vector)
# Test chisquare Op:
chisq_x = f_tt_chisquare(trial_vector)
print("chisquare(0.9*tbme_true) =", chisq_x)

# Manual gradient test:
eps = 1e-2
j_deriv = 30
trial_vector_eps = np.copy(trial_vector)
trial_vector_eps[j_deriv] += eps
# print "trial_vector_eps =", trial_vector_eps
chisq_x_eps = f_tt_chisquare(trial_vector_eps)
print("chisquare(0.9*tbme_true+eps*x_{:d}) =".format(j_deriv), chisq_x_eps)

print("Numerical derivative along axis {:d} = {:f}".format(j_deriv, (chisq_x_eps-chisq_x)/eps))

trial_vector = [ 2.1771    1.0161    2.1771    1.0161    0.2196   -4.54734   0.66096
 -1.02987   1.56807   0.2196   -3.862935 -0.766665  0.766665 -4.54734
  1.59282  -2.622735 -2.34099  -2.04003   1.108791 -2.99583   0.07902
  3.283695 -1.31112  -2.622735 -2.34099   2.04003  -1.108791 -2.99583
 -3.09258   0.07902  -6.54012   0.2196   -4.54734   0.66096  -1.02987
  1.56807  -2.99583   0.07902 ]
chisquare(0.9*tbme_true) = 91.69695205459992
chisquare(0.9*tbme_true+eps*x_30) = 93.23158783839995
Numerical derivative along axis 30 = 153.463578


In [5]:
# Test gradient Op by direct call:
tt_chisquare_grad = ChiSquareGrad(core, model_space, spe_template, tbme_template, nucleiList, KSHELL_DIR,
                                  expLevelsDict, mass_scaling=True, scaling_A0=A0, scaling_p=p, is_mpi=IS_MPI)
f_tt_chisquare_grad = th.function([variable], tt_chisquare_grad(variable))
chisq_grad_x = f_tt_chisquare_grad(trial_vector)
print("chisquareGrad(0.9*tbme_true)[direct chisq eval] =", chisq_grad_x)
# Or instead by grad method of Chisquare:
tt_chisquare_grad = th.tensor.grad(tt_chisquare(variable), variable)
f_tt_chisquare_grad = th.function([variable], tt_chisquare_grad)
chisq_grad_x = f_tt_chisquare_grad(trial_vector)
print("chisquareGrad(0.9*tbme_true)[theano grad method] =", chisq_grad_x)



chisquareGrad(0.9*tbme_true)[direct chisq eval] = [ 1.92458616e+01  9.13586184e+01  1.92458593e+01  9.15844207e+01
  6.43482694e-01  3.14700388e+00  2.66964830e+01  4.49358896e+01
 -2.54781301e+00  1.11569841e-01  1.76115967e+01  4.81556861e+00
 -4.81556813e+00  2.47166946e+00 -6.26799258e+00  2.89067393e+01
  4.55352927e+01  6.85383630e-01 -1.87743972e+00  2.01152827e+01
  1.04367141e+02 -4.41162571e+00  1.18110836e+00  2.89067351e+01
  4.55352923e+01 -6.85388072e-01  1.87744245e+00  1.97968009e+01
  6.07957302e+01  1.04382568e+02  1.52764516e+02  6.43483316e-01
  3.14700613e+00  2.66964820e+01  4.49358860e+01 -2.54781306e+00
  2.01152835e+01  1.04367145e+02]
chisquareGrad(0.9*tbme_true)[theano grad method] = [ 1.92458616e+01  9.13586184e+01  1.92458593e+01  9.15844207e+01
  6.43482694e-01  3.14700388e+00  2.66964830e+01  4.49358896e+01
 -2.54781301e+00  1.11569841e-01  1.76115967e+01  4.81556861e+00
 -4.81556813e+00  2.47166946e+00 -6.26799258e+00  2.89067393e+01
  4.55352927e+01  6.

In [6]:
# === Set up PyMC3 model: ===
with pm.Model() as model:
  print("Starting PyMC3 model setup")
  Nparams = 38
  lower_spe = 0*np.ones(4)
  upper_spe = 5*np.ones(4)
  lower_tbme = -8*np.ones(34)
  upper_tbme = +8*np.ones(34) # Trying to avoid NUTS getting stuck at zero values of all parameters by de-centering the priors from zero
  lower = np.concatenate((lower_spe, lower_tbme))
  upper = np.concatenate((upper_spe, upper_tbme))
  spe_tbme = pm.Uniform('spe_tbme', lower=lower, upper=upper, shape=Nparams)
  
  def logp(dummy_data):
    # Wrapper for the chisquare function to make it a log-likelihood
    return -0.5*tt_chisquare(spe_tbme)

  x = pm.DensityDist('x', logp, observed=0) # Call the likelihood evaluation with a dummy "observed" since the true data are read directly in the chisquare

Starting PyMC3 model setup


In [10]:
# Find MAP (maximum a posteriori), basically the chisquare max likelihoood estimate.
with model:
    print("Calling find_MAP()")
    MAP = pm.find_MAP() # BFGS should be default optimizer
    print("start =")
    print(start)
    

/usr/local/lib/python3.7/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')


Calling find_MAP()


logp = -100.71, ||grad|| = 0.020382: 100%|██████████| 99/99 [01:14<00:00,  1.32it/s]    

start =
{'spe_tbme': array([ 2.42900878,  1.12562968,  2.38459387,  1.14285056, -0.0760348 ,
       -4.95831797,  0.77865884, -1.14415771, -1.72239568,  0.11403798,
       -4.27502138, -0.91628079, -0.78828106,  4.86276079, -1.77188559,
       -2.92612635, -2.57798807,  2.26843729, -1.24694451, -3.32334739,
        0.09470777, -3.65536113,  1.49764655, -2.89159072, -2.52822216,
        2.25975957, -1.26741065, -3.50283412, -3.4442555 ,  0.06533402,
       -7.2771209 ,  0.74309856, -5.35134143,  0.66697209, -1.23320705,
        1.72802312, -3.16616284,  0.11492821]), 'spe_tbme_interval__': array([-0.05680825, -1.23604056, -0.09239057, -1.21640279, -0.01900927,
       -1.44932724,  0.19528295, -0.288014  , -0.43744363,  0.02851143,
       -1.1925053 , -0.23007983, -0.1977118 ,  1.4109932 , -0.45043582,
       -0.76705227, -0.66830829,  0.58308663, -0.3142981 , -0.88428414,
        0.02367805, -0.98682361,  0.37887986, -0.75710294, -0.6544559 ,
        0.58072831, -0.3195442 , -0.93914606

In [8]:
with model:
    # db = pm.backends.Text('pymc3-results')
    print("Calling PyMC3 sampler")
    # trace = pm.sample(100, trace=db, start=start, step=step, init="jitter+adapt_diag", njobs=2)
    trace = pm.sample(100, start=start, cores=1)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Calling PyMC3 sampler


Sequential sampling (2 chains in 1 job)
NUTS: [spe_tbme]
  1%|          | 5/600 [10:41<23:16:31, 140.83s/it]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

Only one chain was sampled, this makes it impossible to run some convergence checks


In [9]:
# Plotting:

plt.figure(0)
pm.energyplot(trace)
plt.savefig("energyplot.pdf")
plt.figure(1)
pm.traceplot(trace)
plt.savefig("traceplot.pdf")
plt.figure(2)
pm.forestplot(trace)
plt.savefig("forestplot.pdf")
plt.show()